In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras.layers import Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers, optimizers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from csv import reader
import re
import spacy
import pickle
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer()

nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
###preprocessing part, this is common to all three datasets
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

def preprocess_text(text):
    text = remove_tags(text)

    # Remove punctuations and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Single character removal
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)

    # Removing multiple spaces
    text = re.sub(r'\s+', ' ', text)

    return text

In [4]:
UNIQUE_WORD_COUNT = 10000
MAX_SEQUENCE_LENGTH = 64

In [5]:
# importing custom dataset
word_index = imdb.get_word_index()
# print(len(word_index.keys()))


In [6]:
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=UNIQUE_WORD_COUNT)
print(testing_data)
print(testing_targets)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


[list([1, 591, 202, 14, 31, 6, 717, 10, 10, 2, 2, 5, 4, 360, 7, 4, 177, 5760, 394, 354, 4, 123, 9, 1035, 1035, 1035, 10, 10, 13, 92, 124, 89, 488, 7944, 100, 28, 1668, 14, 31, 23, 27, 7479, 29, 220, 468, 8, 124, 14, 286, 170, 8, 157, 46, 5, 27, 239, 16, 179, 2, 38, 32, 25, 7944, 451, 202, 14, 6, 717])
 list([1, 14, 22, 3443, 6, 176, 7, 5063, 88, 12, 2679, 23, 1310, 5, 109, 943, 4, 114, 9, 55, 606, 5, 111, 7, 4, 139, 193, 273, 23, 4, 172, 270, 11, 7216, 2, 4, 8463, 2801, 109, 1603, 21, 4, 22, 3861, 8, 6, 1193, 1330, 10, 10, 4, 105, 987, 35, 841, 2, 19, 861, 1074, 5, 1987, 2, 45, 55, 221, 15, 670, 5304, 526, 14, 1069, 4, 405, 5, 2438, 7, 27, 85, 108, 131, 4, 5045, 5304, 3884, 405, 9, 3523, 133, 5, 50, 13, 104, 51, 66, 166, 14, 22, 157, 9, 4, 530, 239, 34, 8463, 2801, 45, 407, 31, 7, 41, 3778, 105, 21, 59, 299, 12, 38, 950, 5, 4521, 15, 45, 629, 488, 2733, 127, 6, 52, 292, 17, 4, 6936, 185, 132, 1988, 5304, 1799, 488, 2693, 47, 6, 392, 173, 4, 2, 4378, 270, 2352, 4, 1500, 7, 4, 65, 55, 73

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [7]:
def vectorize(sequences, dimension = UNIQUE_WORD_COUNT):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):      
        results[i, sequence] = 1
    return results

In [8]:
print((data[1]))
data = vectorize(data)

[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 4369, 5012, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 3103, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2300, 1523, 5, 647, 4, 116, 9, 35, 8163, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 4901, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 6853, 5, 163, 11, 3215, 2, 4, 1153, 9, 194, 775, 7, 8255, 2, 349, 2637, 148, 605, 2, 8003, 15, 123, 125, 68, 2, 6853, 15, 349, 165, 4362, 98, 5, 4, 228, 9, 43, 2, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 4373, 228, 8255, 5, 2, 656, 245, 2350, 5, 4, 9837, 131, 152, 491, 18, 2, 32, 7464, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95]


In [9]:
targets = np.array(targets).astype("float32")

In [10]:
print(len(data))
test_x = data[:10000]
test_y = targets[:10000]
# valid_x = data[5000:10000]
# valid_y = targets[5000:10000]
train_x = data[10000:]
train_y = targets[10000:]

50000


In [11]:
# print(len(train_x))
# print(len(valid_x))
# print(len(test_x))

In [12]:
def LNN():
    model = models.Sequential()
    
    model.add(layers.Dense(50, activation = "relu", input_shape=(10000, )))
    
    model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
    model.add(layers.Dense(50, activation = "relu"))
    model.add(layers.Dropout(0.5, noise_shape=None, seed=None))
    model.add(layers.Dense(50, activation = "relu"))
    model.add(layers.Dropout(0.7, noise_shape=None, seed=None))
    
    model.add(layers.Dense(1, activation = "sigmoid"))
    model.summary()
    
    model.compile(
     optimizer = optimizers.Adam(learning_rate=0.00005),
     loss = "binary_crossentropy",
     metrics = ["accuracy"]
    )
    
    return model

In [13]:
model_LNN = LNN()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                500050    
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5

In [14]:
results_LNN_train = model_LNN.fit(
 train_x, train_y,
 epochs= 8,
 batch_size = 32,
 validation_data = (test_x, test_y)
)

Epoch 1/8
1250/1250 [==============================] - 7s 3ms/step - loss: 0.6694 - accuracy: 0.5708 - val_loss: 0.5585 - val_accuracy: 0.8186
Epoch 2/8
1250/1250 [==============================] - 4s 3ms/step - loss: 0.4993 - accuracy: 0.7779 - val_loss: 0.3434 - val_accuracy: 0.8748
Epoch 3/8
1250/1250 [==============================] - 4s 3ms/step - loss: 0.3711 - accuracy: 0.8615 - val_loss: 0.2834 - val_accuracy: 0.8885
Epoch 4/8
1250/1250 [==============================] - 4s 3ms/step - loss: 0.3061 - accuracy: 0.8927 - val_loss: 0.2654 - val_accuracy: 0.8923
Epoch 5/8
1250/1250 [==============================] - 4s 3ms/step - loss: 0.2668 - accuracy: 0.9081 - val_loss: 0.2620 - val_accuracy: 0.8941
Epoch 6/8
1250/1250 [==============================] - 4s 3ms/step - loss: 0.2352 - accuracy: 0.9214 - val_loss: 0.2662 - val_accuracy: 0.8965
Epoch 7/8
1250/1250 [==============================] - 4s 3ms/step - loss: 0.2084 - accuracy: 0.9305 - val_loss: 0.2696 - val_accuracy: 0.8961

In [15]:
results_LNN_test = model_LNN.evaluate(test_x, test_y)

313/313 [==============================] - 1s 2ms/step - loss: 0.2772 - accuracy: 0.8963


In [16]:
print(results_LNN_test)

[0.2772058844566345, 0.8963000178337097]


In [17]:
my_dict ={}
actualArray =[]
predictedArray = []
word_index = imdb.get_word_index()
    
with open('TestDataset - Sheet1.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    next(csv_reader)
    for row in csv_reader:
        row[1] = row[1].split(',')
        actualArray.append(row[3])
        my_dict[row[0]] = []
        with open(row[4],'r') as file:
            sentence = file.read()
        sentences = sentence.split('.')
        for sentence in sentences:
            doc = nlp(sentence)
            for ent in doc.ents:
                if ent.text in row[1]:
                    sentence = preprocess_text(sentence)
                    my_dict[row[0]].append(sentence)

# print(my_dict['Tony Stark'])
final_dict = {}

In [21]:
def get_encoded_sentences(character):
    encoded_sentences = []
    for sentence in my_dict[character]:
        words = sentence.split()
        encoded_sentence = []
        for word in words:
            if word in word_index:
                if word_index[word] < 10000:
                    encoded_sentence.append(word_index[word])
                else:
                    encoded_sentence.append(0)    
            else:
                encoded_sentence.append(0)
        encoded_sentences.append(encoded_sentence)
    return encoded_sentences

In [25]:
for character in my_dict:
    # print(my_dict[character])
    # print(character)
    enc_sentences = get_encoded_sentences(character)
    # print(enc_sentences)
    positiveSentences = 0
    negativeSentences = 0
    prediction = model_LNN.predict(vectorize(np.asarray(enc_sentences)))
    # print(prediction)
    for pred in prediction:
        ###write the code to assign sentiment to character here
        ##increment positiveSentences and negativeSentences accordingly
        # print(sentence)
        if (pred>=0.5):
            positiveSentences += 1
        else:
            negativeSentences += 1

    total = positiveSentences + negativeSentences
    ##we have to vary this percentage and record stats
    if negativeSentences >= 0.25*total:
        final_dict[character] = "evil"
    else:
        final_dict[character] = "good"
    predictedArray.append(final_dict[character]) 
    

correct = 0
length = len(actualArray)
for i in range(length):
    if actualArray[i]==predictedArray[i]:
        correct = correct + 1
        # print("went here")

        
accuracy = correct/length 
print("correct predictions are")
print(correct)
print("accuracy is ")
print(accuracy)

print(final_dict)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


correct predictions are
30
accuracy is 
0.5
{'Tony Stark': 'evil', 'Gabbar': 'evil', 'Sauron': 'evil', 'Khilji': 'evil', 'Bhallaladeva': 'evil', 'Amit Shellar': 'evil', 'Gandalf': 'evil', 'Raju': 'evil', 'Mogambo': 'evil', 'Kancha': 'evil', 'Kaal': 'evil', 'Loki': 'evil', 'Thanos': 'evil', 'Peter Parker': 'good', 'Valentine': 'evil', 'Venom': 'evil', 'Otto Octavius': 'evil', 'Scar': 'evil', 'Simba': 'evil', 'Lady Tremaine': 'evil', 'Shere Khan': 'evil', 'Mowgli': 'evil', 'Sid Phillips': 'good', 'Woody': 'evil', 'Evelyn': 'evil', 'Bob': 'evil', 'Dolores Umbridge': 'good', 'Robert Callaghan': 'evil', 'Jafar': 'evil', 'Gaston': 'evil', 'Elsa': 'evil', 'Maleficent': 'evil', 'al Ghul': 'evil', 'Kaecilius': 'evil', 'Strange': 'evil', 'Batman': 'evil', 'Harry': 'evil', 'Amarendra Bahubali': 'evil', 'Bilbo Baggins': 'evil', 'Thor': 'evil', 'Saruman': 'evil', 'Frodo': 'evil', 'Farhan': 'evil', 'Louisa': 'evil', 'Biff Tannen': 'evil', 'Hans Gruber': 'evil', 'Chucky': 'evil', 'Jack Dawson': 'evil

In [ ]:
# model_LNN.save('final_models/LNN/')

In [ ]:
preds_model = np.asarray(model_LNN(test_x))
preds = []
for i in preds_model:
    if i>=0.5:
        preds.append(1)
    else:
        preds.append(0)
preds = np.asarray(preds)
preds

In [ ]:
test_y

In [ ]:
cm = confusion_matrix(test_y,preds)
cm

In [ ]:
import itertools
import numpy as np
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.figure(figsize=(6,6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.margins(x=1, y=1)
#     plt.gcf().subplots_adjust(left=0.15)
    plt.savefig('final_models/LNN/conf_matrix.png')

In [ ]:
plot_confusion_matrix(cm, ["Positive", "Negative"])